# KIM models

This Notebook provides an overview of how KIM models are interfaced into the potentials package.

1. Overview of KIM models in potentials

2. Options for identifying installed KIM models

3. Examples

## 1. Overview of KIM models in potentials

Version 0.3.0 of the potentials package completely reworked the handling of KIM models to better reflect how the models are treated in the KIM database.

Getting KIM models to work in LAMMPS requires that

- KIM be installed alongside LAMMPS and LAMMPS built to interface with it.
- Using the kim-api to download and install any models that you wish to use.

The advantage of KIM models is that they exist as installed code and therefore the computed energies and forces will be the same regardless of the simulator software (e.g. LAMMPS) in which the model is used.  The setup does introduce some additional complexity resulting in KIM needing their own version control, which extends to the models themselves.  Only certain versions of a KIM model are compatible with a version of the kim-api, and only certain versions of the kim-api are compatible with a version of LAMMPS.  Because of this, it is impractical to include all KIM model versions as separate entries and instead the list should only include models that are currently installed with the particular LAMMPS+kim-api that is to be used.

The potentials package handles the KIM models by

- potential_LAMMPS_KIM records are stored in the potentials database.  Each potential_LAMMPS_KIM record is associated with the KIM model's "shortcode", which uniquely identifies what the model represents without any model version details. The potential_LAMMPS_KIM records tie the model to any associated Potential record entries, and list the full ids for all known versions of the model.
- When LAMMPS potentials are loaded, entries for the installed KIM models are generated by comparing the list of installed KIM model ids to those found in the potential_LAMMPS_KIM records. 



## 2. Options for identifying installed KIM models

As described above, only entries for installed KIM models will be generated when LAMMPS potentials are loaded.  The manner in which the list of installed KIM models is controlled by parameter settings when a potentials.Database object is created, and default values set using potentials.Settings.

### 2.1 Indentifying installed KIM models when initializing a Database

When initializing a Database object, the following parameters give three different options for how the installed KIM models are identified

- __kim_api_directory__ (*path-like object, optional*) is the root directory of the installed KIM API version that you plan on using for the simulations.  If given, the associated kim-api-collections-management will be called to determine the list of installed KIM models.
- __installed_kim_models__ (*list, optional*) is a list of full KIM ids.
- __kim_settings_file__ (*path-like object, optional*) is the name of a JSON/XML file that contains subelement 'installed-kim-models' that lists full KIM ids.

While giving kim_api_directory ensures that the listed KIM models are always consistent with the installed models, specifying the lists separately using either of the other two parameters can be useful if multiple kim-api versions are to be used or if the kim-api to be used is inaccessible at the moment.

Default Database initialization with no parameters (and no values set using settings)

In [ ]:
potdb = potentials.Database()
potdb.installed_kim_models

Specifying the kim_api_directory during Database initilization

In [ ]:
potdb = potentials.Database(kim_api_directory='/home/lmh1/lammps/lammps/lib/kim/installed-2.1.3')
potdb.installed_kim_models

Specifying the installed_kim_models during Database initilization

In [ ]:
installed_kim_models = [
    '',
    ''
]

potdb = potentials.Database(installed_kim_models=installed_kim_models)
potdb.installed_kim_models

Specifying a kim_settings_file during Database initialization

In [ ]:
kim_settings = {}
kim_settings['installed-kim-models'] = [
    '',
    '',
    ''
]

with open('kim_settings.json') as f:
    json.dump(kim_settings, fp=f)
    
    
potdb = potentials.Database(kim_settings_file='kim_settings.json')
potdb.installed_kim_models

### 2.2. Finding, setting, saving, and loading installed KIM models after Database initialization

The Database class has additional supporting methods for further control of the list of installed KIM models.  Using the find, set, or load operations after a Database has initialized behave similarly to using the initialization parameters mentioned above.

#### find_installed_kim_models()

Uses the kim api to discover the installed KIM models.

Parameters
    
- __kim_api_directory__ (*path-like object, optional*) The path to the directory associated with the kim api version to use to build the list of installed models.

#### set_installed_kim_models()

Allows for the list of installed KIM models to be directly set.  Useful if the kim api is on a different machine or if the list should include models associated with different kim api versions.

Parameters

- __value__ (*str or list*) The list of installed kim models to use.

#### save_installed_kim_models()

Saves the list of installed kim models so that they can be retrieved later. The list of models will be saved in "installed_kim_models.json" in the settings directory unless filename is given.

Parameters
    
- __filename__ (*path-like object, optional*) Allows for the list of installed kim models to be saved to a different file.  Will be saved in JSON format

#### load_installed_kim_models()

Loads the list of installed kim models.

Parameters
    
- __kim_settings_file__ (*path-like object, optional*) The path to a json file with an 'installed-kim-models' field that lists the installed kim models.  If not given, will access "installed_kim_models.json" in the settings directory.

### 2.3. Setting default KIM options

If save_installed_kim_models() is called without specifying a filename, then the list will be saved inside the settings directory.  Then, the next time a Database is initialized without any of the kim parameters listed in section 2.1. the values in the default settings file will be read in.

In [ ]:
print('Currently identified installed KIM models:')
print(potdb.installed_kim_models)

# Save values to the default setting
potdb.save_installed_kim_models()

# Initialize a new database object without any parameters to show it loaded the default list
newpotdb = potentials.Database()
print(newpotdb.installed_kim_models)

A default KIM API directory value can also be set using potentials.Settings(), which will automatically be used when the Database class is initialized if the kim parameters listed in section 2.1 are not given.

__NOTE__: If both the default kim_api_directory value and the default list inside the settings directory exist, then the explicit list is used over the kim_api_directory value.  To work around this, you can either

- Call find_installed_kim_models() after initializing the Database and give it no parameters,
- Delete the default kim settings file. The Database attribute default_kim_settings_file gives its file path, or
- Specify kim_api_directory when initializing a Database and then call save_installed_kim_models() to overwrite the list.

In [ ]:
print('Current default kim_api_directory is:')
print(potentials.Settings().kim_api_directory)

# Delete the list of installed models from settings if it exists
if potdb.default_kim_settings_file.exists():
    potdb.default_kim_settings_file.unlink()

# Set the default kim_api_directory
potentials.Settings().set_kim_api_directory('/home/lmh1/lammps/lammps/lib/kim/installed-2.1.3')

print('New default kim_api_directory is:')
print(potentials.Settings().kim_api_directory)

# Initialize a new database object without any parameters to show it used the default kim_api_directory
newpotdb = potentials.Database()
print(newpotdb.installed_kim_models)

## 3. KIM model usage